# Ce notebook rassemble les méthodes statistiques employées pour la sélection de variables pour le modèle de régression logistique ordinale

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from mord import LogisticAT
from typing import List
import numpy as np

import pickle

In [2]:
df_train = pd.read_csv("../../data/train.csv")
df_test = pd.read_csv("../../data/test.csv")

In [3]:
X_train = df_train.drop(labels=["score"], axis=1).values
y_train = df_train["score"].values

X_test = df_test.drop(labels=["score"], axis=1).values
y_test = df_test["score"].values

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Méthodes de sélection de variables

## *1-* Sélection par régression Lasso

LogisticRegressionCV est utilisé pour effectuer une régression logistique avec une validation croisée pour sélectionner le meilleur paramètre de régularisation (C) parmi une liste de valeurs candidates ()

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from mord import LogisticAT

# modèle de régression logistique avec pénalité L1 (LASSO)
model_lasso = LogisticRegression(penalty='l1', solver='saga', max_iter=1000, random_state=42)

# SelectFromModel pour effectuer la sélection de variables
sfm = SelectFromModel(estimator=model_lasso)
sfm.fit(X_train, y_train)

# indices des variables sélectionnées
indices_lasso = sfm.get_support(indices=True)
variables = [df_train.columns[i] for i in indices_lasso]
# coefficients des variables sélectionnées
coefficients_lasso = sfm.estimator_.coef_.flatten()

# noms et coefficients des variables
print("#########################################################")
print()
print("Coefficients obtenus après Lasso:")
for variable, coefficient in zip(variables, coefficients_lasso):
    print(f"{variable}: {coefficient}")

# On pénalise en ne prenant que les coeffs différents de 0 (on pourrait prendre >0.05 aussi)
indices_choisis = [i for i, coef in zip(indices_lasso, coefficients_lasso) if coef != 0]

variables_choisies = [df_train.columns[i] for i in indices_choisis]

print("##########################################################")
print()
print("Variables sélectionnées après Lasso")
print()
for variable in variables_choisies:
    print(f"{variable}")


# on utilise les indices filtrés pour obtenir les données des variables optimales sélectionnées
X_train_final = X_train[:, indices_choisis]
X_test_final = X_test[:, indices_choisis]

# Maintenant, utilisez les variables sélectionnées pour la régression logistique ordinale avec mord
model_ordinal = LogisticAT(alpha=0)  # Utilisez alpha=0 pour désactiver la régularisation (ou ajustez selon vos besoins)

# Entraînez le modèle sur les données d'entraînement avec les variables sélectionnées
model_ordinal.fit(X_train_final, y_train)

# Faites des prédictions sur les données de test avec les variables sélectionnées
predictions_ordinal = model_ordinal.predict(X_test_final)

# Calculez l'exactitude du modèle de régression logistique ordinale
accuracy_ordinal = accuracy_score(y_test, predictions_ordinal)

print()
print("Exactitude du modèle de régression logistique ordinale : {:.2f}%".format(accuracy_ordinal * 100))
print()

print("#####################################")
print()
print("Enregistrement du modèle de régression logistique")
with open('../model/modele_ordinal.pkl', 'wb') as model_file:
    pickle.dump(model_ordinal, model_file)

print("Modèle ordinal enregistré")


#########################################################

Coefficients obtenus après Lasso:
energy_100g: 0.0
fat_100g: 0.2312759394181233
saturated-fat_100g: 2.020391817385482
trans-fat_100g: 0.8203335309026503
cholesterol_100g: 1.251817937524018
carbohydrates_100g: 0.0
sugars_100g: 1.6820724367932676
fiber_100g: -0.847604921358526
proteins_100g: 0.09164171790418724
salt_100g: 0.5031184929786225
sodium_100g: 0.5023665030859524
vitamin-a_100g: 0.3783509923197762
vitamin-c_100g: 0.31518011118007916
calcium_100g: -0.05431301772506332
iron_100g: 0.0
nutrition-score-fr_100g: 13.76347393595731
code: 0.20051679584593599
##########################################################

Variables sélectionnées après Lasso

fat_100g
saturated-fat_100g
trans-fat_100g
cholesterol_100g
sugars_100g
fiber_100g
proteins_100g
salt_100g
sodium_100g
vitamin-a_100g
vitamin-c_100g
calcium_100g
nutrition-score-fr_100g
code

Exactitude du modèle de régression logistique ordinale : 100.00%

#######################

c:\Users\jaure\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


### Re-test du modèle enregistré

In [6]:
with open('../model/modele_ordinal.pkl', 'rb') as model_file:
    model_ordinal = pickle.load(model_file)
    
predictions = model_ordinal.predict(X_test_final)
accuracy_ordinal = accuracy_score(y_test, predictions)
print("{:.2f}%".format(accuracy_ordinal * 100))

100.00%
